# Resep.in — Model Training (Versioned)

Train a **YOLOv8n** food ingredient detector with full experiment tracking.

| Component | Tool | What it tracks |
|-----------|------|----------------|
| Dataset | DVC | Images, labels, data.yaml (via `.dvc` pointer files in Git) |
| Experiments | MLflow | Hyperparameters, metrics, artifacts per training run |
| Model Registry | MLflow | Production / Staging model versions |
| Config | `version_config.py` | Dataset version, class list, paths |

In [ ]:
# Install / upgrade Ultralytics (YOLOv8)
!pip install ultralytics --upgrade --force-reinstall

  Using cached matplotlib-3.10.8-cp312-cp312-win_amd64.whl.metadata (52 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.9.1-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp312-cp312-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd6

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.2.11 requires idna==3.7, but you have idna 3.11 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.9.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Install PyTorch with CUDA 11.8 support (adjust URL for your CUDA version)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/2.8 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 GB 16.4 MB/s eta 0:02:52
   ---------------------------------------- 0.0/2.8 GB 3.0 MB/s eta 0:15:53
   ---------------------------------------- 0.0/2.8 GB 2.4 MB/s eta 0:19:36
   ---------------------------------------- 0.0/2.8 GB 2.3 MB/s eta 0:20:03
   ---------------------------------------- 0.0/2.8 GB 2.2 MB/s eta 0:21:20
   ---------------------------------------- 0.0/2.8 GB 2.1 MB/s eta 0:22:03
   ---------------------------------------- 0.0/2.8 GB 2.1 MB/s eta 0:22:38
   ---------------------------------------- 0.0/2.8 GB 2.0 MB/s eta 0:23:01
   ---------------------------------------- 0.0/2.8 GB 2.1 MB/s eta 0:22:46
   ---------------------------------------- 0.0/2.8 GB 2.0 MB/s eta 0:22:53
   ---------------------------------------- 0.0/2


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Import Model & Check GPU

In [ ]:
import torch
from ultralytics import YOLO

print("YOLO imported successfully")
print(f"GPU available: {torch.cuda.is_available()}")

model = YOLO("yolov8n.pt")

suskses import YOLO
GPU available: True


In [ ]:
import torch
import sys

print(f"1. Python Version: {sys.version.split()[0]}")
print(f"2. PyTorch Version: {torch.__version__}")
print(f"3. CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"4. GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"5. CUDA Version: {torch.version.cuda}")

    try:
        x = torch.ones(1).cuda()
        print("6. GPU Tensor Test: PASSED")
    except Exception as e:
        print(f"6. GPU Tensor Test: FAILED ({e})")
else:
    print("GPU NOT DETECTED. PyTorch may be installed without CUDA support.")

1. Python Version: 3.12.9
2. PyTorch Version: 2.12.0.dev20260218+cu128
3. CUDA Available: True
4. GPU Name: NVIDIA GeForce RTX 5070 Laptop GPU
5. CUDA Version di PyTorch: 12.8
6. Test Tensor ke GPU: BERHASIL ✅


## 2. Dataset Statistics

In [ ]:
import os

dataset_path = "C:/resep.in/dataset_normalisasi_fix"


def count_images(split_name):
    """Count image files (jpg/jpeg/png) in a dataset split."""
    img_dir = os.path.join(dataset_path, split_name, "images")
    if not os.path.exists(img_dir):
        return 0
    return len([f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))])


n_train = count_images("train")
n_val = count_images("valid") or count_images("val")
n_test = count_images("test")
total = n_train + n_val + n_test

print("=== DATASET STATISTICS ===")
print(f"Train : {n_train:,} images")
print(f"Valid : {n_val:,} images")
print(f"Test  : {n_test:,} images")
print(f"{'─' * 28}")
print(f"Total : {total:,} images")

if total > 0:
    print(f"\nSplit Ratio:")
    print(f"  Train : {n_train / total * 100:.1f}%")
    print(f"  Valid : {n_val / total * 100:.1f}%")
    print(f"  Test  : {n_test / total * 100:.1f}%")
else:
    print("Dataset not found. Check dataset_path above.")

## 3. Training

In [1]:
# === VERSIONED TRAINING ===
# This cell uses the versioned training pipeline with MLflow tracking.
# All hyperparameters, metrics, and weights are automatically logged.

from train_versioned import TrainingConfig, run_training

# Configure training (modify parameters as needed)
config = TrainingConfig(
    model_name="yolov8n.pt",
    epochs=50,
    patience=3,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=0.001,
    device="0",
)

# Run training with full versioning
result = run_training(config)

# Print summary
print(f"\nRun ID: {result['run_id']}")
print(f"Run Name: {result['run_name']}")
print(f"Results Dir: {result['results_dir']}")
print(f"mAP50: {result['metrics'].get('mAP50', 'N/A')}")
print(f"mAP50-95: {result['metrics'].get('mAP50_95', 'N/A')}")

c:\Users\Shallom Otniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  VERSIONED TRAINING: v1.0.0_yolov8n_20260225_092135
Loaded existing manifest: v1.0.0
MLflow tracking URI: sqlite:///c:/resep.in/mlflow.db
MLflow experiment: resep-in-food-detection (ID: 1)

MLflow Run ID: 2c4ffcc4cca044e09fe60c03fe3d3f72

Loading model: yolov8n.pt
Starting training...
New https://pypi.org/project/ultralytics/8.4.16 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.14  Python-3.12.9 torch-2.12.0.dev20260218+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\resep.in\dataset_normalisasi_fix\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=

## 4. Post-Training: Model Promotion & MLflow UI

After training completes:
1. **View MLflow UI** — run `mlflow ui` in terminal, then open http://localhost:5000
2. **Promote model** — if the new run outperforms the current production model
3. **Compare runs** — use MLflow UI to compare metrics across training runs

In [18]:
import importlib
import promote_model
importlib.reload(promote_model)
from promote_model import promote_model

promote_model(
    run_id=result["run_id"],
    metric_name="mAP50_95",
    min_threshold=0.15,
    description=f"Dataset v{result['run_name']}",
)

MLflow tracking URI: sqlite:///c:/resep.in/mlflow.db
MLflow experiment: resep-in-food-detection (ID: 1)

Candidate model (run 2c4ffcc4...):
  mAP50_95: 0.6924
Registered model version 4 from run 2c4ffcc4cca044e09fe60c03fe3d3f72

Current production model (v1):
  mAP50_95: 0.6924

Candidate does not outperform production. Registered as v4 (staging).


False

In [19]:
# === LIST ALL MODEL VERSIONS ===
from promote_model import list_models
list_models()

MLflow tracking URI: sqlite:///c:/resep.in/mlflow.db
MLflow experiment: resep-in-food-detection (ID: 1)

Version    Stage           mAP50      mAP50-95   Run ID
----------------------------------------------------------------------
v1         none            0.8965     0.6924     2c4ffcc4...
v2         none            0.8965     0.6924     2c4ffcc4...
v3         none            0.8965     0.6924     2c4ffcc4...
v4         none            0.8965     0.6924     2c4ffcc4...
